## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
!pip install pyspark

     |████████████████████████████████| 217.8 MB 4.2 kB/s  eta 0:00:01
     |████████████████████████████████| 197 kB 57.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218258791 sha256=65256614dc0f851c8d2bd15ec02858e133cc8a15047678da8fa17d6b89f4d502
  Stored in directory: /root/.cache/pip/wheels/84/30/e3/c51c5cd0229631e662d29d7b578a3e5949a4c8db033ffb70aa
Successfully built pyspark


In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [9]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [10]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([2.25805779e-01, 1.89462936e+00, 2.94969941e+00, 2.44716475e+00,
       1.46460774e+00, 2.85797028e+00, 3.59386220e+00, 3.23525641e+00,
       2.68554794e-01, 5.74409246e+00, 5.11237835e+00, 7.81621799e+00,
       4.74753338e+00, 3.92967553e+00, 5.08830104e+00, 9.16930767e+00,
       6.13264400e+00, 8.18431153e+00, 1.84755599e+01, 1.99312105e+01,
       7.44688706e+00, 3.07661657e+00, 2.04198685e+01, 3.06691099e+00,
       1.92877953e+01, 2.28999785e+01, 2.64419145e+01, 2.03868975e+01,
       9.39775768e-01, 3.39634715e+00, 2.11618380e+01, 2.88553577e+01,
       2.66072743e+01, 2.45535866e+01, 1.89940502e+01, 1.33926723e+01,
       1.83589363e+01, 3.32870414e+01, 3.03078831e+01, 2.30239019e+01,
       6.03497932e+00, 3.80200582e+01, 2.84709325e+01, 1.19620403e+01,
       2.13660264e+01, 1.36030709e+01, 3.99453374e+01, 2.44537641e+01,
       1.05348585e+01, 3.60536762e+00, 1.73930440e+01, 4.97961134e+01,
       1.66011894e+00, 1.18785553e+01, 2.99993194e+01, 3.64662809e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [12]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.top(5)


[980.823592792312,
 971.3948586230476,
 970.8592727164388,
 939.9488573195991,
 926.8819770941924]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num*0.92


revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [14]:
# perform action to retrieve rdd values
revenue_minus_tax.take(5)

[0.20774131701514184,
 1.7430590148898137,
 2.713723459285386,
 2.2513915716716713,
 1.3474391237422565]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [16]:
discounted = revenue_minus_tax.map(lambda x: x * 0.9)

In [17]:
discounted.take(10)

[0.18696718531362766,
 1.5687531134008323,
 2.4423511133568474,
 2.026252414504504,
 1.212695211368031,
 2.3663993919339914,
 2.975717904292941,
 2.678792303856909,
 0.22236336928748612,
 4.7561085588903005]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [18]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[812.1219348320343,
 804.3149429398835,
 803.8714778092113,
 778.2776538606282,
 767.4582770339914,
 742.288243360158,
 731.0442212951788,
 722.2254569035871,
 719.2533443091893,
 715.3111006389199,
 707.5543312418679,
 704.2666066875238,
 704.1778927112641,
 689.1212038444793,
 686.4647826102836]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [19]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [20]:
mapped =price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.2258057793642846, 0.18696718531362766), (1.894629364010667, 1.5687531134008323), (2.9496994122667237, 2.4423511133568474), (2.447164751817034, 2.026252414504504), (1.4646077431981048, 1.212695211368031), (2.857970280113516, 2.3663993919339914), (3.5938622032523435, 2.975717904292941), (3.235256405624286, 2.678792303856909), (0.2685547938254663, 0.22236336928748612), (5.744092462427899, 4.7561085588903005)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [21]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.2258057793642846, 0.18696718531362766, 1.894629364010667, 1.5687531134008323, 2.9496994122667237, 2.4423511133568474, 2.447164751817034, 2.026252414504504, 1.4646077431981048, 1.212695211368031]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [24]:
# use the filter function
selected_items = discounted.filter(lambda x: x >= 300)

# calculate total remaining in inventory 
selected_items.count()

270

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [27]:
selected_items.reduce(lambda x, y: x + y)


127012.48899977715

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [28]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(43, 314.28653766954216),
 (45, 310.38411578550597),
 (5, 350.16233104064463),
 (25, 345.98207939721215),
 (43, 323.872155090259),
 (45, 374.7315341387163),
 (20, 373.0539651114126)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [29]:
# calculate how much each user spent
total = sales_data.reduceByKey(lambda x, y: x + y)
total.take(10)

[(20, 3354.5251627340176),
 (40, 1308.85967650884),
 (10, 2981.7892454764274),
 (50, 2413.0529003313254),
 (31, 7617.197279002593),
 (21, 2076.524548047369),
 (41, 4571.086239408885),
 (1, 4118.4742048255),
 (11, 1145.2564582174973),
 (12, 2543.9712307578047)]

In [30]:
# sort the users from highest to lowest spenders
total.sortBy(lambda x: x[1],ascending = False).collect()

[(31, 7617.197279002593),
 (45, 6877.737767410961),
 (24, 6140.795771902406),
 (38, 5742.363285353347),
 (35, 5184.126399399749),
 (26, 5096.226277652709),
 (37, 4886.6162585728),
 (4, 4649.695880861098),
 (41, 4571.086239408885),
 (33, 4325.375652045734),
 (1, 4118.4742048255),
 (43, 4053.541989507151),
 (13, 3928.029611206846),
 (6, 3507.0154221706584),
 (20, 3354.5251627340176),
 (9, 3351.1211640821716),
 (7, 3273.595710221937),
 (10, 2981.7892454764274),
 (5, 2781.4542639919605),
 (12, 2543.9712307578047),
 (42, 2543.331040179324),
 (18, 2481.826692377889),
 (17, 2425.2873059610592),
 (50, 2413.0529003313254),
 (3, 2255.3500169308845),
 (48, 2136.061215900014),
 (21, 2076.524548047369),
 (14, 2008.6229163595763),
 (36, 2003.1299459988522),
 (39, 1934.5813740224798),
 (29, 1866.136302011549),
 (25, 1555.2528864880924),
 (44, 1551.4771980579603),
 (27, 1488.8392372038052),
 (40, 1308.85967650884),
 (16, 1221.5608536025857),
 (46, 1154.9541509860255),
 (11, 1145.2564582174973),
 (19, 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [ ]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
